In [ ]:
import xarray as xr
import numpy as np

accum_path   = "../data/waterTransport-accum.nc"
instant_path = "../data/waterTransport-instant.nc"

ds_accum   = xr.open_dataset(accum_path)
ds_instant = xr.open_dataset(instant_path)

ds_accum, ds_instant


(<xarray.Dataset> Size: 12GB
 Dimensions:     (valid_time: 1464, latitude: 721, longitude: 1440)
 Coordinates:
   * valid_time  (valid_time) datetime64[ns] 12kB 2021-11-01 ... 2021-12-31T23...
     expver      (valid_time) <U4 23kB ...
   * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
   * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
     number      int64 8B ...
 Data variables:
     tp          (valid_time, latitude, longitude) float32 6GB ...
     e           (valid_time, latitude, longitude) float32 6GB ...
 Attributes:
     GRIB_centre:             ecmf
     GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
     GRIB_subCentre:          0
     Conventions:             CF-1.7
     institution:             European Centre for Medium-Range Weather Forecasts
     history:                 2026-02-10T00:34 GRIB to CDM+CF via cfgrib-0.9.1...,
 <xarray.Dataset> Size: 6GB
 Dimensions:     (valid_time: 14

In [3]:
print(ds_accum.data_vars)
print(ds_instant.data_vars)


Data variables:
    tp       (valid_time, latitude, longitude) float32 6GB ...
    e        (valid_time, latitude, longitude) float32 6GB ...
Data variables:
    tcw      (valid_time, latitude, longitude) float32 6GB ...


In [4]:
# show a small corner + a few timesteps for each var
for name in ["tp", "e"]:
    print("\n", name)
    print(ds_accum[name].isel(valid_time=slice(0, 2), latitude=slice(0, 3), longitude=slice(0, 5)).values)

print("\n tcw")
print(ds_instant["tcw"].isel(valid_time=slice(0, 2), latitude=slice(0, 3), longitude=slice(0, 5)).values)



 tp
[[[8.5830688e-06 8.5830688e-06 8.5830688e-06 8.5830688e-06 8.5830688e-06]
  [8.5830688e-06 8.5830688e-06 8.5830688e-06 8.5830688e-06 8.5830688e-06]
  [8.5830688e-06 8.5830688e-06 8.5830688e-06 8.5830688e-06 8.5830688e-06]]

 [[1.3828278e-05 1.3828278e-05 1.3828278e-05 1.3828278e-05 1.3828278e-05]
  [8.5830688e-06 8.5830688e-06 8.5830688e-06 8.5830688e-06 8.5830688e-06]
  [8.5830688e-06 8.5830688e-06 8.5830688e-06 8.5830688e-06 8.5830688e-06]]]

 e
[[[-5.6887511e-06 -5.6887511e-06 -5.6887511e-06 -5.6887511e-06
   -5.6887511e-06]
  [-8.1027392e-06 -8.1027392e-06 -8.1027392e-06 -8.1027392e-06
   -8.1027392e-06]
  [-7.3278788e-06 -7.2980765e-06 -7.2980765e-06 -7.2980765e-06
   -7.2980765e-06]]

 [[-6.7143701e-06 -6.7143701e-06 -6.7143701e-06 -6.7143701e-06
   -6.7143701e-06]
  [-9.7840093e-06 -9.7840093e-06 -9.7840093e-06 -9.7542070e-06
   -9.7542070e-06]
  [-9.8138116e-06 -9.7840093e-06 -9.7840093e-06 -9.7542070e-06
   -9.7542070e-06]]]

 tcw
[[[3.8292766 3.8292766 3.8292766 3.829276

In [25]:
i = 1000  # first date

tp_max  = float(ds_accum["tp"].isel(valid_time=i).max().values)
e_min   = float(ds_accum["e"].isel(valid_time=i).min().values)   # most negative (strongest evap)
e_max   = float(ds_accum["e"].isel(valid_time=i).max().values)
evap_max = float(np.maximum(-ds_accum["e"].isel(valid_time=i).values, 0.0).max())

tcw_max = float(ds_instant["tcw"].isel(valid_time=i).max().values)

print("tp max (m):", tp_max)
print("e min (m):", e_min)
print("e max (m):", e_max)
print("evap max (m):", evap_max)
print("tcw max:", tcw_max)


tp max (m): 0.021343231201171875
e min (m): -0.0010091415606439114
e max (m): 0.00023170793429017067
evap max (m): 0.0010091415606439114
tcw max: 90.66205596923828


In [5]:
print("tp chunks:", ds_accum["tp"].encoding.get("chunksizes"))
print("e  chunks:", ds_accum["e"].encoding.get("chunksizes"))
print("tcw chunks:", ds_instant["tcw"].encoding.get("chunksizes"))

tp chunks: (183, 91, 180)
e  chunks: (183, 91, 180)
tcw chunks: (183, 91, 180)


In [ ]:
# Cell 2 — elper
import numpy as np
import os
import imageio.v2 as imageio

OUT_DIR = "../data/waterTransport-evap-precip-waterColumn"
os.makedirs(OUT_DIR, exist_ok=True)

def to_u8(x, vmin, vmax):
    y = (x - vmin) / (vmax - vmin)
    np.clip(y, 0.0, 1.0, out=y)
    return (y * 255.0).astype(np.uint8)


In [29]:
i = 0  # change this

tp  = ds_accum["tp"].isel(valid_time=i).values
e   = ds_accum["e"].isel(valid_time=i).values
tcw = ds_instant["tcw"].isel(valid_time=i).values

evap = np.maximum(-e, 0.0)

In [45]:
TP_MIN,  TP_MAX   = 0.0, 0.02
EVAP_MIN,EVAP_MAX = 0.0, 0.003
TCW_MIN, TCW_MAX  = 0.0, 110.0

r = to_u8(tp,   TP_MIN,   TP_MAX)
g = to_u8(evap, EVAP_MIN, EVAP_MAX)
b = to_u8(tcw,  TCW_MIN,  TCW_MAX)

rgb = np.stack([r, g, b], axis=-1)

ts = np.datetime_as_string(ds_accum["valid_time"].values[i], unit="s").replace(":", "-")
out_path = os.path.join(OUT_DIR, f"{ts}.png")
imageio.imwrite(out_path, rgb)

out_path

'../data/waterTransport-evap-precip-waterColumn/2021-11-01T00-00-00.png'

In [1]:
import xarray as xr

path = "../data/tcw_1981_2020_mean.nc"
ds = xr.open_dataset(path)

ds  # in Jupyter, this displays a nice summary


<xarray.Dataset> Size: 2GB
Dimensions:     (valid_time: 480, latitude: 721, longitude: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 4kB 1981-01-01 ... 2020-12-01
    expver      (valid_time) <U4 8kB ...
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    number      int64 8B ...
Data variables:
    tcw         (valid_time, latitude, longitude) float32 2GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2026-02-10T21:10 GRIB to CDM+CF via cfgrib-0.9.1...

In [2]:
tname = "valid_time" if "valid_time" in ds.coords else ("time" if "time" in ds.coords else None)
print("time coord:", tname)

if tname is None:
    print("coords:", list(ds.coords))
else:
    t = ds[tname]
    print("n_time:", t.size)
    print("first:", t.values[0], "last:", t.values[-1])
    print("unique months:", sorted(set(t.dt.month.values.tolist())))
    print("first 20 timestamps:\n", t.values[:20])


time coord: valid_time
n_time: 480
first: 1981-01-01T00:00:00.000000000 last: 2020-12-01T00:00:00.000000000
unique months: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
first 20 timestamps:
 ['1981-01-01T00:00:00.000000000' '1981-02-01T00:00:00.000000000'
 '1981-03-01T00:00:00.000000000' '1981-04-01T00:00:00.000000000'
 '1981-05-01T00:00:00.000000000' '1981-06-01T00:00:00.000000000'
 '1981-07-01T00:00:00.000000000' '1981-08-01T00:00:00.000000000'
 '1981-09-01T00:00:00.000000000' '1981-10-01T00:00:00.000000000'
 '1981-11-01T00:00:00.000000000' '1981-12-01T00:00:00.000000000'
 '1982-01-01T00:00:00.000000000' '1982-02-01T00:00:00.000000000'
 '1982-03-01T00:00:00.000000000' '1982-04-01T00:00:00.000000000'
 '1982-05-01T00:00:00.000000000' '1982-06-01T00:00:00.000000000'
 '1982-07-01T00:00:00.000000000' '1982-08-01T00:00:00.000000000']


In [3]:
print("data vars:", list(ds.data_vars))
v = "tcw" if "tcw" in ds.data_vars else list(ds.data_vars)[0]
print("chosen var:", v)
print(ds[v])


data vars: ['tcw']
chosen var: tcw
<xarray.DataArray 'tcw' (valid_time: 480, latitude: 721, longitude: 1440)> Size: 2GB
[498355200 values with dtype=float32]
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 4kB 1981-01-01 ... 2020-12-01
    expver      (valid_time) <U4 8kB ...
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    number      int64 8B ...
Attributes: (12/32)
    GRIB_paramId:                             136
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      1038240
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            avgua
    ...                                       ...
    GRIB_totalNumber:                         0
    GRIB_units:                               kg m**-2
    long_name:                                